
 # Dataset:

## user related features
cms_segid: 0-96

cms_group_id: 0-12

age: 0-6

pvalue: 1,2,3

shopping_level:1,2,3

occupation: 1,0

## from ad (onsite features)
cate_id: 6725

adgroup_id: 827009

brand:98772

customer:252841

price: float

In [19]:
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import pandas_profiling 
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from iopath.common.file_io import PathManager	
from iopath.fb.manifold import ManifoldPathHandler
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from sklearn.model_selection import train_test_split	
import itertools
from sklearn import metrics
from tqdm import tqdm

In [20]:
if torch.cuda.is_available():
    device = "cuda"
    #torch.set_default_tensor_type("torch.cuda.FloatTensor")
    print("USING CUDA")
else:
    device = "cpu"
    print("USING CPU")

# sending model to cuda has error (not sure why)
#device = 'cpu'

USING CUDA


In [21]:
def load_taobao_df():
    path_manager = PathManager()
    path_manager.register_handler(ManifoldPathHandler(has_user_data=False))
    ad = "manifold://fsl/tree/taobao/ad_feature.csv"
    ad_path = path_manager.open(ad, "rb")
    ad_feature_df =  pd.read_csv(ad_path)

    raw_sample = "manifold://fsl/tree/taobao/raw_sample.csv"
    raw_sample_path = path_manager.open(raw_sample, "rb")
    raw_sample_df = pd.read_csv(raw_sample_path)

    user = "manifold://fsl/tree/taobao/user_profile.csv"
    user_path = path_manager.open(user, "rb")
    user_profile_df = pd.read_csv(user_path)

    # memory optimize for ad feature dataframe
    optimized_gl = raw_sample_df.copy()

    gl_int = raw_sample_df.select_dtypes(include=['int'])
    converted_int = gl_int.apply(pd.to_numeric,downcast='unsigned')
    optimized_gl[converted_int.columns] = converted_int


    gl_obj = raw_sample_df.select_dtypes(include=['object']).copy()
    converted_obj = pd.DataFrame()
    for col in gl_obj.columns:
        num_unique_values = len(gl_obj[col].unique())
        num_total_values = len(gl_obj[col])
        if num_unique_values / num_total_values < 0.5:
            converted_obj.loc[:,col] = gl_obj[col].astype('category')
        else:
            converted_obj.loc[:,col] = gl_obj[col]
    optimized_gl[converted_obj.columns] = converted_obj
    raw_sample_df = optimized_gl.copy()
    raw_sample_df_new = raw_sample_df.rename(columns = {"user": "userid"})
    optimized_g2 = ad_feature_df.copy()
    g2_int = ad_feature_df.select_dtypes(include=['int'])
    converted_int = g2_int.apply(pd.to_numeric,downcast='unsigned')
    optimized_g2[converted_int.columns] = converted_int

    g2_float = ad_feature_df.select_dtypes(include=['float'])
    converted_float = g2_float.apply(pd.to_numeric,downcast='float')
    optimized_g2[converted_float.columns] = converted_float

    optimized_g3 = user_profile_df.copy()

    g3_int = user_profile_df.select_dtypes(include=['int'])
    converted_int = g3_int.apply(pd.to_numeric,downcast='unsigned')
    optimized_g3[converted_int.columns] = converted_int

    g3_float = user_profile_df.select_dtypes(include=['float'])
    converted_float = g3_float.apply(pd.to_numeric,downcast='float')
    optimized_g3[converted_float.columns] = converted_float

    # combine 3 tables
    df1 = raw_sample_df_new.merge(optimized_g3, on="userid")
    final_df = df1.merge(optimized_g2, on="adgroup_id")

    final_df['pvalue_level'] = final_df['pvalue_level'].fillna(2, )
    final_df['final_gender_code'] = final_df['final_gender_code'].fillna(1, )
    final_df['age_level'] = final_df['age_level'].fillna(3, )
    final_df['shopping_level'] = final_df['shopping_level'].fillna(2, )
    final_df['occupation'] = final_df['occupation'].fillna(0, )
    final_df['brand'] = final_df['brand'].fillna(0, )
    final_df['customer'] = final_df['customer'].fillna(0, )
    final_df['cms_group_id'] = final_df['cms_group_id'].fillna(13, )

    final_df['pvalue_level'] -= 1
    final_df['shopping_level'] -= 1
    final_df = final_df.astype({"cms_segid": int, 
                                "cms_group_id": int, 
                                'clk': float, 
                                'adgroup_id': int, 
                                'final_gender_code':int,
                                'age_level':int,
                                'pvalue_level':int,
                                'shopping_level':int,
                                'occupation':int,
                                'cate_id':int,
                                'customer':int,
                                'brand':int}
                                )

    return final_df

In [22]:
tb_df = load_taobao_df()

In [23]:
# checking the dataset
# how many label 1 
# so that the dataset is very unbalanced
sum(tb_df['clk'])/len(tb_df)

0.051320095719300095

In [24]:
# can we get a balanced dataset, by taking the same number of positive label and negative labels
# so that training would be ok for now
df_1 = tb_df[tb_df['clk'] == 1]
df_0 = tb_df[tb_df['clk'] == 0]
print(len(df_1))
print(len(df_0))
frames = [df_1, df_0[:1284513]]
df = pd.concat(frames)
print(len(df))
df = df.sample(frac=1)
tb_train = df[:50000]
tb_test = df[-10000:]
tb_test = tb_test.reset_index()
tb_train = tb_train.reset_index()

1284513
23744922


2569026


In [25]:
# train test split
#tb_train = tb_df[:2252650] # 90% for training
#tb_test = tb_df[2252650:]
#tb_test = tb_test.reset_index()

# cpu slow, so just test a few 
#tb_train = tb_df[:50000] # 90% for training
#tb_test = tb_df[-10000:]
#tb_test = tb_test.reset_index()

## Model: server_arch and client_arch

In [26]:
class Dataset_split(Dataset):
    def __init__(self, X, emb_cols, server_continuous_cols, non_emb_cols):
        X = X.copy()
        self.server_categorical = X.loc[:,emb_cols].copy().values.astype(np.int64) #categorical columns
        self.client = X.reindex(columns = non_emb_cols).values.astype(np.float32)
        self.server_continuous = X.reindex(columns = server_continuous_cols).values.astype(np.float32)
        self.y = X['clk']
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        #return self.server_categorical[idx], self.server_continuous[idx], self.client[idx], self.y[idx]

        return {'server_categorical': self.server_categorical[idx],
                'server_continuous': self.server_continuous[idx],
                'client': self.client[idx],
                'label': self.y[idx]}

Create 2 parts of model: server_arch and client_arch like in FSL project. 

The server_arch takes in 2 types of input: categorical and continuous. The categorical input will go through the embedding layer first and then concatenate with the continuous input. There is only 1 continuous features 'price' in this dataset.

The client_arch takes in only one input. In our case, all 5 features are categorical, which are user related and we are going to reconstructed these 5 features. 

In [27]:
# server side arch
class server_arch(nn.Module):
    def __init__(self, embedding_sizes, n_cont):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories,size in embedding_sizes])
        n_emb = sum(e.embedding_dim for e in self.embeddings) #length of all embeddings combined
        self.n_emb, self.n_cont = 250, n_cont
        self.emb_drop = nn.Dropout(0.6)
        self.lin1 = nn.Linear(self.n_emb + self.n_cont, 64)
        

    def forward(self, x_server_categorical,x_server_continuous):
        x = [e(x_server_categorical[:,i]) for i,e in enumerate(self.embeddings)]
        x = torch.cat(x, 1)
        x = torch.cat([x,x_server_continuous], 1)
        x = self.emb_drop(x)
        x = F.relu(self.lin1(x))
        return x

class client_arch(nn.Module):
    def __init__(self, n_server_output, n_client_input):
        super().__init__()
        self.n_server_output = n_server_output
        self.n_client_input = n_client_input
        self.lin1 = nn.Linear(self.n_server_output + self.n_client_input, 32)
        self.lin2 = nn.Linear(32, 32)
        self.lin3 = nn.Linear(32, 1)
        #self.bn1 = nn.BatchNorm1d(self.n_cont)
        #self.bn2 = nn.BatchNorm1d(64)
        #self.bn3 = nn.BatchNorm1d(32)
        #self.emb_drop = nn.Dropout(0.6)
        #self.drops = nn.Dropout(0.3)
        

    def forward(self, server_output, x_client):
        cut_layer = torch.cat([server_output, x_client], 1)
        x = F.relu(self.lin1(cut_layer))
        #x = self.drops(x)
        #x = self.bn2(x)
        x = F.relu(self.lin2(x))
        #x = self.drops(x)
        #x = self.bn3(x)
        x = torch.sigmoid(self.lin3(x))
        return x
    
    def set_model_gradients(self, gradient):
        params = list(self.parameters())
        for p, grad in zip(params, gradient):
            p.grad = grad.clone()

    def backward(self, cut_node, output, target, loss_fn, update_model_grad=True):
        loss = loss_fn(output, target)
        params = list(self.parameters())
                
        # gradients dE/dw_n
        de_dw = torch.autograd.grad(
            [loss],
            params,
            retain_graph = True,
        )

        if update_model_grad:
            self.set_model_gradients(de_dw)


        de_dx = torch.autograd.grad(
            [loss],
            cut_node,
            retain_graph = True,
        )

        return de_dw, de_dx


## fit() and validate() function

In [28]:
def validate_split(server_model, client_model, loss_fn, test_dataloader, device):
    server_model.eval()
    client_model.eval()
    total = 0
    sum_loss = 0
    correct = 0
    y_proba_list = []
    y_true_list = []
    y_pred_list = []
    for batch in test_dataloader:
        for key, value in batch.items():
                batch[key] = batch[key].to(device)

        y = batch['label'].unsqueeze(1).to(torch.float32)

        current_batch_size = y.shape[0]

        server_output = server_model(batch['server_categorical'], batch['server_continuous'])
        out = client_model(server_output, batch['client'])
        
        y_proba_list.extend(out.detach().cpu().numpy())
        y_true_list.extend(y.detach().cpu().numpy())

        loss = loss_fn(out, y)
        sum_loss += current_batch_size*(loss.item())
        total += current_batch_size
        pred = torch.round(out).squeeze()

        y_pred_list.extend(pred.detach().cpu().numpy())
        y = y.squeeze()
        correct += (pred == y).sum()

        

    print("test loss %.3f and accuracy %.3f" % (sum_loss/total, correct/total))
    auc = metrics.average_precision_score(y_true_list,y_proba_list)
    print("AUC")
    print(auc)
    conf_matrix = metrics.confusion_matrix(y_true_list,y_pred_list)
    print('confusion matrix')
    print(conf_matrix)
    return sum_loss/total, correct/total

In [29]:
# build training loop that for separate model
def one_round_split_train(server_model, client_model, server_opt, client_opt, batch , batch_idx, loss_fn):
    # zero all optimizers
    server_opt.zero_grad()
    client_opt.zero_grad()

    server_output = server_model(batch['server_categorical'],batch['server_continuous'])
    client_output = client_model(server_output,batch['client'])

    label = batch['label'].unsqueeze(1).to(torch.float32)

    loss = loss_fn(client_output, label)

    # backward
    dw, dx = client_model.backward(server_output, client_output, label,loss_fn, update_model_grad=True)

    server_output.backward(dx)
    server_opt.step()
    client_opt.step()
    return loss, client_output     

In [30]:
def fit(server_model, client_model, trainloader, testloader, epochs, lr= 1, momentum=0):
    loss_fn = nn.BCELoss()
    server_opt = torch.optim.SGD(server_model.parameters(),lr=lr, momentum = momentum)
    client_opt = torch.optim.SGD(client_model.parameters(),lr=lr, momentum = momentum)

    batch_idx = 0
    total_loss = 0
    total = 0
    print('starting fiting')
    for e in range(0,epochs):
        print('epoch', e)
        for batch in trainloader:
            batch_idx += 1
            #print(batch_idx)
            for key, value in batch.items():
                batch[key] = batch[key].to(device)

            loss, output = one_round_split_train(server_model, client_model, server_opt, client_opt, batch, batch_idx, loss_fn)
            total_loss += loss
            total += batch['label'].shape[0]

            if batch_idx % 500 == 0:
                val_loss, val_acc = validate_split(server_model, client_model, loss_fn, testloader, device)

## Training

In [31]:
#creating train and valid datasets
client_cols = ['cms_group_id','age_level','pvalue_level','shopping_level','occupation']
embedding_cols = ['adgroup_id','cate_id','customer','brand','cms_segid']
continuous = ['price']
trainset = Dataset_split(tb_train, embedding_cols, continuous, client_cols)
testset = Dataset_split(tb_test, embedding_cols, continuous, client_cols)
print(len(trainset))
print(len(testset))

50000
10000


In [32]:
batch_size = 32
train_dataloader = DataLoader(trainset, batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(testset, batch_size=batch_size,shuffle=True)

# fitting
epochs = 2
lr = 0.01
momentum = 0
device = 'cpu'
embedded_cols = {'adgroup_id':846811,
            'cate_id': 12960,
            'customer': 255875,
            'brand': 461497,
            'cms_segid': 96}
embedding_sizes = [(846811+1, 50), (12960+1,50), (255875+1,50),(461497+1,50),(96+1,50)]
client_cols = ['cms_group_id','age_level','pvalue_level','shopping_level','occupation']
server_continous = ['price']
server_model = server_arch(embedding_sizes, 1).to(device)
client_model = client_arch(64, len(client_cols)).to(device)

fit(server_model, client_model,train_dataloader, test_dataloader, epochs, lr, momentum)

starting fiting
epoch 0


test loss 0.687 and accuracy 0.536
AUC
0.6089683804519889
confusion matrix
[[4276  609]
 [4034 1081]]


test loss 0.680 and accuracy 0.590
AUC
0.6254590467321528
confusion matrix
[[3410 1475]
 [2630 2485]]


test loss 0.684 and accuracy 0.535
AUC
0.6245446014900116
confusion matrix
[[4233  652]
 [3998 1117]]


epoch 1


test loss 0.680 and accuracy 0.594
AUC
0.6359430916442365
confusion matrix
[[3789 1096]
 [2966 2149]]


test loss 0.679 and accuracy 0.577
AUC
0.6396847834433075
confusion matrix
[[4068  817]
 [3416 1699]]


test loss 0.675 and accuracy 0.582
AUC
0.6460644597865597
confusion matrix
[[3894  991]
 [3185 1930]]


## Attack

In [33]:
class Reconstruct_taobao:
    def __init__(self, server_model, client_model):
        self.server_model = server_model
        self.client_model = client_model

    def reconstruct(
        self,
        cluster_combination_list,
        server_output,
        label,
        dx_original,
        cate_original,
        offsite_feature_shape=5,
    ):
        self.client_model.eval()
        #cluster_combination_list = self.get_cluster_combination(cluster_centers)
        dx_diff_list = []

        for cate_try in cluster_combination_list:
            # forward pass the client side model
            # use the other data, but the reconstructed cluster center to be offsite feature to see if dx match.
            cate_try = torch.tensor(cate_try).type(torch.float32)
            cate_try = cate_try.unsqueeze(0)
            
            client_output_recon = client_output = client_model(server_output,cate_try)

            # backward pass the client side model
            # get the gradient and compare with dx and store
            
            dw_recon, dx_recon = client_model.backward(
                            server_output, 
                            client_output_recon, 
                            label,
                            loss_fn, 
                            update_model_grad=False)

            dx_recon = dx_recon[0]
            # calculate the distance between dx and dx_original
            # to get the smallest distance as the reconstructed center
            dx_diff = self.get_dist_between_dx(dx_recon, dx_original)
            dx_diff_list.append(dx_diff.clone().detach())

        # output the lowest distance cluster center
        minpos = dx_diff_list.index(min(dx_diff_list))
        reconstruct_output = cluster_combination_list[minpos]

        # check for accuracy
        # assert center_original in cluster_combination_list
        cate_original = cate_original[0].int()

        reconstruct_output = torch.tensor(reconstruct_output)

        # get the acc for each features
        acc = [int(i == j) for i, j in zip(reconstruct_output, cate_original)]
        
        return acc

    def get_dist_between_dx(self, dx_recon, dx_original):
        pdist = torch.nn.PairwiseDistance(p=2)
        pdistloss = torch.mean(pdist(dx_recon, dx_original))
        return pdistloss

In [34]:
# start doing the attack
# testloader too to have batch 1
def attack_cluster(testloader, cluster_combination_list, server_model, client_model, loss_fn, num_features=5):
    stats_output= {}
    acc_total = [0 for i in range(num_features)]
    num_samples = 0
    # initiaze the reconstruct machine
    recon_machine = Reconstruct_taobao(
        server_model = server_model,
        client_model = client_model
    )

    server_model.eval()
    client_model.eval()

    for batch_idx, batch in enumerate(testloader):
        num_samples += 1
        if batch_idx < 4500:
            continue
        cate_original = batch['client']
        
        #print('cate original', cate_original.size())
        label = batch['label'].unsqueeze(1).to(torch.float32)

        server_output_original = server_model(batch['server_categorical'],batch['server_continuous'])
        client_output_original = client_model(server_output_original,batch['client'])

        dw_original, dx_original = client_model.backward(
                server_output_original, 
                client_output_original, 
                label,
                loss_fn, 
                update_model_grad=False)
        dx_original = dx_original[0]

        # doing the attack
        acc = recon_machine.reconstruct(
                    cluster_combination_list,
                    server_output_original,
                    label,
                    dx_original,
                    cate_original,
        )
        
        acc_total = [sum(i) for i in zip(acc_total, acc)]  

        if batch_idx % 500 == 0:
            print('batch idx', batch_idx)
            print('acc_total', acc_total)
            print('acc per features', [x / num_samples for x in acc_total])
            print('acc average over features', sum(acc_total)/(num_samples*num_features))
        
    stats_ouput['acc_per_features'] = [x / num_samples for x in acc_total]
    stats_output['acc_avg'] = sum(acc_total)/(num_samples*num_features)
    print(stats_ouput['acc'] )
    return stats_output


In [35]:
test_dataloader_attack = DataLoader(testset, batch_size=1,shuffle=False)
loss_fn = nn.BCELoss()
# features can be attacked
feature_offsite = ['cms_group_id','age_level','pvalue_level','shopping_level','occupation']
offsite_category = [13, 7, 3, 3, 2] 
cluster = []

# create a combination
for i in range(len(offsite_category)):
    temp = [j for j in range(offsite_category[i])]
    cluster.append(temp)

cluster_combination_list = list(itertools.product(*cluster))
print('total number of diffierent combination of category:', len(cluster_combination_list))

total number of diffierent combination of category: 1638


In [36]:
# run the attack
output = attack_cluster(test_dataloader_attack, cluster_combination_list, server_model, client_model, loss_fn)

batch idx 4500
acc_total [1, 1, 1, 1, 1]
acc per features [0.00022217285047767163, 0.00022217285047767163, 0.00022217285047767163, 0.00022217285047767163, 0.00022217285047767163]
acc average over features 0.00022217285047767163


batch idx 5000
acc_total [454, 464, 466, 470, 478]
acc per features [0.09078184363127374, 0.09278144371125775, 0.09318136372725455, 0.09398120375924815, 0.09558088382323535]
acc average over features 0.09326134773045391


batch idx 5500
acc_total [904, 912, 934, 933, 946]
acc per features [0.1643337574986366, 0.16578803853844756, 0.16978731139792766, 0.16960552626795128, 0.17196873295764406]
acc average over features 0.16829667333212142


batch idx 6000
acc_total [1362, 1374, 1392, 1396, 1419]
acc per features [0.22696217297117147, 0.22896183969338443, 0.23196133977670388, 0.23262789535077488, 0.23646058990168306]
acc average over features 0.23139476753874355


batch idx 6500
acc_total [1813, 1832, 1857, 1865, 1889]
acc per features [0.2788801722811875, 0.28180279956929705, 0.28564836179049374, 0.2868789417012767, 0.2905706814336256]
acc average over features 0.2847561913551761


...

batch idx 7500
acc_total [2709, 2747, 2772, 2793, 2835]
acc per features [0.36115184642047726, 0.36621783762165044, 0.3695507265697907, 0.3723503532862285, 0.3779496067191041]
acc average over features 0.3694440741234502


batch idx 8000
acc_total [3154, 3202, 3237, 3246, 3300]
acc per features [0.39420072490938635, 0.4001999750031246, 0.40457442819647543, 0.4056992875890514, 0.4124484439445069]
acc average over features 0.40342457192850895


batch idx 8500
acc_total [3608, 3662, 3699, 3712, 3773]
acc per features [0.4244206563933655, 0.4307728502529114, 0.4351252793788966, 0.4366545112339725, 0.443830137630867]
acc average over features 0.4341606869780026


batch idx 9000
acc_total [4063, 4121, 4163, 4174, 4252]
acc per features [0.4513942895233863, 0.4578380179980002, 0.4625041662037551, 0.46372625263859574, 0.4723919564492834]
acc average over features 0.4615709365626042


batch idx 9500
acc_total [4520, 4589, 4629, 4642, 4729]
acc per features [0.4757393958530681, 0.4830017892853384, 0.4872118724344806, 0.4885801494579518, 0.49773708030733604]
acc average over features 0.486454057467635


In [2]:
a = [4501, 4499, 4495, 4496, 4500]
b = [4520, 4589, 4629, 4642, 4729]
c = [(a+b)/9500 for a, b in zip(a,b)]
c

[0.9495789473684211,
 0.9566315789473684,
 0.960421052631579,
 0.9618947368421052,
 0.9714736842105263]